In [35]:
import json
import pandas as pd
import numpy as np
import requests

ACCOUNT_ID = "001-001-2265866-001"
API_TOKEN = "591691d91a571232b46b9b87e74145ba-4fc5bc3427b4eb7d79c3501316fdffa0"

API_URL = "api-fxtrade.oanda.com"
STREAM_URL = "stream-fxtrade.oanda.com"

PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"

query = {"instruments": "EUR_USD"} 
headers = {"Authorization": "Bearer "+ API_TOKEN}

response = requests.get("https://"+API_URL+PRICING_PATH, headers=headers, params=query) 

response.json()

{'time': '2024-08-05T03:03:42.109643248Z',
 'prices': [{'type': 'PRICE',
   'time': '2024-08-05T03:03:41.035879122Z',
   'bids': [{'price': '1.09230', 'liquidity': 500000},
    {'price': '1.09229', 'liquidity': 2500000},
    {'price': '1.09228', 'liquidity': 2000000},
    {'price': '1.09227', 'liquidity': 5000000},
    {'price': '1.09224', 'liquidity': 10000000},
    {'price': '1.09221', 'liquidity': 10000000}],
   'asks': [{'price': '1.09246', 'liquidity': 500000},
    {'price': '1.09247', 'liquidity': 500000},
    {'price': '1.09248', 'liquidity': 2000000},
    {'price': '1.09249', 'liquidity': 2000000},
    {'price': '1.09250', 'liquidity': 5000000},
    {'price': '1.09252', 'liquidity': 10000000},
    {'price': '1.09255', 'liquidity': 10000000}],
   'closeoutBid': '1.09221',
   'closeoutAsk': '1.09255',
   'status': 'tradeable',
   'tradeable': True,
   'quoteHomeConversionFactors': {'positiveUnits': '1.00000000',
    'negativeUnits': '1.00000000'},
   'instrument': 'EUR_USD'}]}

In [36]:
from OandaAPI import OandaAPI
oa = OandaAPI(account_id=ACCOUNT_ID, api_token=API_TOKEN)

gbp_jpy_df = oa.show_granularity()
shit_df = oa.historical_data()

gbp_jpy_df

KeyError: 'candles'

In [37]:
ACCOUNT_ID = "001-001-2265866-001"
API_TOKEN = "591691d91a571232b46b9b87e74145ba-4fc5bc3427b4eb7d79c3501316fdffa0"

API_URL = "api-fxtrade.oanda.com"
STREAM_URL = "stream-fxtrade.oanda.com"

PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"

def get_historical_data(currency_pair='EUR_USD', start_date='08/09/2023', end_date='08/09/2024', granularity='D' ,as_pandas=True):
    """Will get the historical json data for a given currency pair.

    Args:
        currency_pair: Currency pair name. 
        start_date: Date of start of historical data.
        end_date: Closest date of historical data. 
        granularity: How often price data is updated.
        as_pandas: Set to True to return as pandas dataframe.

    Returns:
        json: Historical pricing data in json format.
        Pandas: Historical pricing dataframe.
    """
    # Must import Necessary Libraries
    import time
    import requests
    import pandas as pd

    # Convert Dates to DateTime objects
    start_dateTime = pd.to_datetime(start_date)
    end_dateTime = pd.to_datetime(end_date)

    # Convert Start Date & End Date to a UNIX timestamp
    unix_start = time.mktime(start_dateTime.timetuple())
    unix_end = time.mktime(end_dateTime.timetuple())

    # Authorization Header
    auth_header = {"Authorization": "Bearer "+API_TOKEN} 

    # Query Parameters
    parameters = {"from": str(unix_start), 
                    "to": str(unix_end), 
                    "granularity": granularity}
    
    # Historical Candle Stick Data URL
    candle_url = f"/v3/accounts/{ACCOUNT_ID}/instruments/{currency_pair}/candles"

    # Request for Candle Stick Data
    json_response = requests.get("https://"+API_URL+candle_url, headers=auth_header, params=parameters).json()

    # Create Pandas DataFrame
    if as_pandas is True:
        # Convert Json to Pandas df
        candle_df = pd.json_normalize(json_response["candles"])

        # Drop complete Column
        candle_df.drop('complete', inplace=True, axis=1)

        # Rename Columns
        candle_df.rename(inplace=True,columns={'time':'DateTime','volume':'Volume','mid.o':'Open',
                                                'mid.h':'High','mid.l':'Low','mid.c':'Close'})

        # Convert Datetime= Column to datetime object
        candle_df['DateTime'] = pd.to_datetime(candle_df['DateTime'])

        # Set DateTime to Index
        candle_df.set_index('DateTime',inplace=True)

        # Restructure Column Names
        candle_df = candle_df[['Open','High','Low','Close','Volume']]
        
        # Return DataFrame
        return candle_df
    
    # Return as Json 
    else: 
        return json_response

df = get_historical_data(currency_pair='EUR_USD', 
                     start_date='08/01/2023',
                    end_date='08/01/2024',
                    granularity='H2',
                    as_pandas=True)
df

,Open,High,Low,Close,Volume
DateTime,,,,,
2023-08-01 05:00:00+00:00,1.09904,1.10004,1.09868,1.09948,7560
2023-08-01 07:00:00+00:00,1.09944,1.09949,1.09702,1.09748,11225
2023-08-01 09:00:00+00:00,1.09749,1.09847,1.09658,1.09822,8147
2023-08-01 11:00:00+00:00,1.09822,1.09824,1.09524,1.09565,10231
2023-08-01 13:00:00+00:00,1.09566,1.09892,1.09560,1.09783,16453
...,...,...,...,...,...
2024-07-31 19:00:00+00:00,1.08265,1.08329,1.08170,1.08264,10399
2024-07-31 21:00:00+00:00,1.08248,1.08310,1.08241,1.08243,906
2024-07-31 23:00:00+00:00,1.08246,1.08301,1.08218,1.08277,7084


In [ ]:
import time    
from datetime import datetime, timedelta
import math
def get_all_candles(start,end,gran):

    gran_secs = {'S5': 5,'S10': 10,'S15': 15,'S30': 30,
                 'M1': 60,'M2': 120,'M4': 240,'M5': 300,
                 'M10': 600,'M15': 900,'M30': 1800,
                 'H1': 3600,'H2': 7200,'H3': 10800,
                 'H4': 14400,'H6': 21600,'H8': 28800,
                 'H12': 43200,'D': 86400,'W': 604800,
                 'M': 2620800}

    print(f'start Date:{start} & end Date:{end}')
    start_dateTime = pd.to_datetime(start)
    end_dateTime = pd.to_datetime(end)
    print(f'start DateTime:{start_dateTime} & end DateTime:{end_dateTime}')


    time_distance = end_dateTime - start_dateTime   
    seconds_between = time_distance.total_seconds() 
    num_data_points = seconds_between / gran_secs.get(gran)
    print(f'Seconds Between:{seconds_between}')


    if num_data_points > 4000:
        print('Greater than 4000 seconds')
        max_date = start_dateTime+timedelta(seconds=4000*math.floor(gran_secs.get(gran)))
        print(f'First Max date: {max_date}')

        # Create Pandas DataFrame
        can_df = pd.DataFrame()

        new_start_dateTime = start_dateTime
       
        while True:
            if max_date >= end_dateTime:
                print(f'{new_start_dateTime} - {end_dateTime}')
                new_df = test_get_historical_data(instrument='EUR_USD', 
                                            start_date=new_start_dateTime,
                                            end_date=end_dateTime,
                                            granularity=gran)
                can_df = pd.concat([can_df, new_df])
                break

            else:
                print(f'{new_start_dateTime} - {max_date}')
                new_df = test_get_historical_data(instrument='EUR_USD', 
                                                start_date=new_start_dateTime,
                                                end_date=max_date,
                                                granularity=gran)
                new_start_dateTime = max_date+timedelta(seconds=gran_secs.get(gran))
                max_date = new_start_dateTime+timedelta(seconds=4000*math.floor(gran_secs.get(gran)))
                can_df = pd.concat([can_df, new_df])
                                        
        return can_df
    else: 
        print(f'{start_dateTime} - {end_dateTime}')
        can_df = test_get_historical_data(instrument='EUR_USD', 
                                    start_date=start_dateTime,
                                    end_date=end_dateTime,
                                    granularity=gran)

        return can_df

master_df = get_all_candles(start='08/01/2005',
                            end='08/01/2024',
                            gran='M5')

master_df

In [20]:
import time
from datetime import datetime, timedelta
import math
import tqdm
def get_all_candles_upgraded(start,end,gran, verbose=False, currency_pair='EUR_USD'):

    gran_secs = {'S5': 5,'S10': 10,'S15': 15,'S30': 30,
                 'M1': 60,'M2': 120,'M4': 240,'M5': 300,
                 'M10': 600,'M15': 900,'M30': 1800,
                 'H1': 3600,'H2': 7200,'H3': 10800,
                 'H4': 14400,'H6': 21600,'H8': 28800,
                 'H12': 43200,'D': 86400,'W': 604800,
                 'M': 2620800}

    MAX_CANDLES = 4000

    start_dateTime = pd.to_datetime(start)
    end_dateTime = pd.to_datetime(end)
    print(f'Gathering Data {start_dateTime} - {end_dateTime}')


    
    time_distance = end_dateTime - start_dateTime   
    seconds_between = time_distance.total_seconds() 
    num_candles = seconds_between / gran_secs.get(gran)
    print(f'Seconds Between:{seconds_between}')

    if num_candles > MAX_CANDLES:
        max_date = start_dateTime+timedelta(seconds=MAX_CANDLES*math.floor(gran_secs.get(gran)))

        # Create Pandas DataFrame
        can_df = pd.DataFrame()
        start_dateTime_list = []
        end_dateTime_list = []

        new_start_dateTime = start_dateTime

        while True:
            if max_date >= end_dateTime:
                start_dateTime_list.append(new_start_dateTime)
                end_dateTime_list.append(end_dateTime)
                print(f'Last Line: {new_start_dateTime} - {end_dateTime}')
                break

            else:
                # print(f'{new_start_dateTime} - {max_date}')
                start_dateTime_list.append(new_start_dateTime)
                end_dateTime_list.append(max_date) 
                
                new_start_dateTime = max_date+timedelta(seconds=gran_secs.get(gran))
                max_date = new_start_dateTime+timedelta(seconds=4000*math.floor(gran_secs.get(gran)))

        for i in tqdm.tqdm(range(len(start_dateTime_list)),colour='GREEN', 
                           ncols=100,desc =f'Downloading {currency_pair} {gran} Data'):

            new_df = historical_data(currency_pair=currency_pair, 
                                    start_date=start_dateTime_list[i],
                                    end_date=end_dateTime_list[i],
                                    granularity=gran)
            
            can_df = pd.concat([can_df, new_df])
        
        return can_df

    else:
        new_df = historical_data(currency_pair=currency_pair, 
                                          start_date=start_dateTime_list[i],
                                          end_date=end_dateTime_list[i],
                                          granularity=gran)


master_df = get_all_candles_upgraded(start='08/01/2024',
                                     end='08/02/2024',
                                     gran='M1',
                                     pair='EUR_USD')

master_df

TypeError: get_all_candles_upgraded() got an unexpected keyword argument 'pair'

In [27]:
def candle_delta(start_date='08/09/2023', end_date='08/09/2024', granularity='M5') -> float:
        """This will return the number of candles between start_date and end_date.

        Args:
            start_date: Date to start calculations.
            end_date: Date if ending.
            granularity: Price update frequency.

        Returns:
            float: Number of candle sticks between start_date and end_date.
        """
        # Import Necessary Libraries
        import pandas as pd

        # Convert Dates to dateTimes
        start_dateTime = pd.to_datetime(start_date)
        end_dateTime = pd.to_datetime(end_date)

        # Calculate Seconds Between DateTimes
        datetime_dif = end_dateTime - start_dateTime   
        seconds_between = datetime_dif.total_seconds() 

        # Convert second between to number of candles
        num_candles = seconds_between / gran_sec_dict.get(granularity)

        # Return Number of Candle Sicks
        return num_candles

In [29]:
print(candle_delta(granularity='S10'))
gran_secs = {'S5': 5,'S10': 10,'S15': 15,'S30': 30,
                'M1': 60,'M2': 120,'M4': 240,'M5': 300,
                'M10': 600,'M15': 900,'M30': 1800,
                'H1': 3600,'H2': 7200,'H3': 10800,
                'H4': 14400,'H6': 21600,'H8': 28800,
                'H12': 43200,'D': 86400,'W': 604800,
                'M': 2620800}

3162240.0


In [30]:
MAX_CANDLES = 4000
start_date = '08/09/2023'

max_date = start_date+timedelta(seconds=MAX_CANDLES*math.floor(gran_secs.get('M5')))

print(max_date)

TypeError: can only concatenate str (not "datetime.timedelta") to str

In [8]:
ACCOUNT_ID = "001-001-2265866-001"
API_TOKEN = "591691d91a571232b46b9b87e74145ba-4fc5bc3427b4eb7d79c3501316fdffa0"
API_URL = "api-fxtrade.oanda.com"

from OandaAPI import OandaAPI
oa = OandaAPI(account_id=ACCOUNT_ID, api_token=API_TOKEN)

In [16]:
def historical_data(currency_pair='EUR_USD', start_date='08/09/2023', end_date='08/09/2024', granularity='D', as_pandas=True):
        """Will get the historical json data for a given currency pair.

        Args:
            currency_pair: Currency pair name. 
            start_date: Date of start of historical data.
            end_date: Closest date of historical data. 
            granularity: How often price data is updated.
            as_pandas: Set to True to return as pandas dataframe.

        Returns:
            json: Historical pricing data in json format.
            Pandas: Historical pricing dataframe.
        """
        # Must import Necessary Libraries
        import time
        import requests
        import pandas as pd

        # Convert Dates to DateTime objects
        start_dateTime = pd.to_datetime(start_date)
        end_dateTime = pd.to_datetime(end_date)

        # Convert Start Date & End Date to a UNIX timestamp
        unix_start = time.mktime(start_dateTime.timetuple())
        unix_end = time.mktime(end_dateTime.timetuple())

        # Authorization Header
        auth_header = {"Authorization": "Bearer "+API_TOKEN} 

        # Query Parameters
        parameters = {"from": str(unix_start), 
                        "to": str(unix_end), 
                        "granularity": granularity}
        
        # Historical Candle Stick Data URL
        candle_url = f"/v3/accounts/{ACCOUNT_ID}/instruments/{currency_pair}/candles"

        # Request for Candle Stick Data
        json_response = requests.get("https://"+API_URL+candle_url, headers=auth_header, params=parameters).json()

        # Create Pandas DataFrame
        if as_pandas is True:
            # Convert Json to Pandas df
            candle_df = pd.json_normalize(json_response["candles"])

            # Drop complete Column
            candle_df.drop('complete', inplace=True, axis=1)

            # Rename Columns
            candle_df.rename(inplace=True,columns={'time':'DateTime','volume':'Volume','mid.o':'Open',
                                                    'mid.h':'High','mid.l':'Low','mid.c':'Close'})

            # Convert Datetime= Column to datetime object
            candle_df['DateTime'] = pd.to_datetime(candle_df['DateTime'])

            # Set DateTime to Index
            candle_df.set_index('DateTime',inplace=True)

            # Restructure Column Names
            candle_df = candle_df[['Open','High','Low','Close','Volume']]
            
            # Return DataFrame
            return candle_df
        
        # Return as Json 
        else: 
            return json_response

In [17]:
this_df = historical_data(currency_pair='EUR_USD', start_date='08/09/2020',
                                                  end_date='08/01/2020', granularity='D')

this_df

KeyError: 'candles'

In [3]:
practice_candle_df = oa.load_candlestick_datafame(currency_pair='EUR_USD', start_date='08/09/2020',
                                                  end_date='08/01/2020', granularity='D')

practice_candle_df

KeyError: 'candles'